In [2]:
import gkeepapi
from pprint import pprint as print
keep = gkeepapi.Keep()
success = keep.login('achalagarwal.01@gmail.com', 'kvlatubhkxhlldsi')

In [72]:
import subprocess
import re
from logging import Logger

In [4]:
# proc = subprocess.Popen(['task', 'add', 'test_child' ], stderr=subprocess.PIPE, stdout=subprocess.PIPE)
# stdout_value = proc.communicate()
# print('\tstdout:' + str(stdout_value))

In [5]:
#string = stdout_value[0].decode('ascii')

In [6]:
#re.findall(r'\d+', string )

In [7]:
# proc = subprocess.Popen(['task', '23', 'delete' ], stderr=subprocess.PIPE, stdout=subprocess.PIPE, stdin=subprocess.PIPE)
# # proc.stdin()
# yes = bytes('yes \n', 'ascii')
# stdout_value = proc.communicate(yes)
# # stdout_value = proc.communicate(b'all \n')
# print('\tstdout:' + str(stdout_value))

In [8]:
# string = stdout_value[0].decode('ascii')

In [9]:
# re.findall(r'Deleted [1-9]+ task.', string) 

In [10]:
def add_to_taskwarrior(task, parent=None, project='Google Keep'):

    # add task in taskwarrior
    proc = subprocess.Popen(['task', 'add', task['text'] ], stderr=subprocess.PIPE, stdout=subprocess.PIPE)
    stdout_value = proc.communicate()

    # extract taskwarrior id from stdout
    string = stdout_value[0].decode('ascii')
    task_id = re.findall(r'\d+', string )[0]
    task['task_id'] = task_id

    # now we annotate the google keep task id
    subprocess.Popen(['task', task['task_id'], 'annotate', task['keep_id']]).communicate()

    # add as a dependent to parent
    if parent:
        proc = subprocess.Popen(['task', parent, 'modify', 'depends:'+task['task_id'] ], stderr=subprocess.PIPE, stdout=subprocess.PIPE, stdin=subprocess.PIPE)
        stdout_value = proc.communicate()

    # iterate for all children
    for child in task['children']:
        add_to_taskwarrior(child, parent=task['task_id'], project=project)


In [11]:
def delete_from_taskwarrior(tasks):

    # create arguments depending on whether tasks is a single number or a list of numbers
    # TODO Should we enforce string input here?
    task_string, confirm_string, count = (','.join([str(task) for task in tasks]), 'all \n', len(tasks)) if hasattr(tasks, '__iter__') else (str(tasks), 'yes \n', 1)

    # delete tasks from taskwarrior
    proc = subprocess.Popen(['task', task_string, 'delete' ], stderr=subprocess.PIPE, stdout=subprocess.PIPE, stdin=subprocess.PIPE)

    # confirm deletion of all tasks provided
    stdout_value = proc.communicate(bytes(confirm_string, 'ascii'))
    string = stdout_value[0].decode('ascii')
    return True if re.findall(rf'Deleted {count}+ task.', string) else False
        


In [12]:
# proc = subprocess.Popen(['task', '25', 'modify', 'depends:26' ], stderr=subprocess.PIPE, stdout=subprocess.PIPE, stdin=subprocess.PIPE)
# # proc.stdin()
# stdout_value = proc.communicate()

# # stdout_value = proc.communicate(b'yes \n')

# print('\tstdout:' + str(stdout_value))

In [13]:
def has_attr_value(obj, attr='', val=''):
    return getattr(obj, attr) == val

In [44]:
notes = keep.all()

In [45]:
filtered_notes = list(filter(lambda x: has_attr_value(x, 'pinned', True), notes))

In [46]:
filtered_notes

In [47]:
filtered_notes = list(filter(lambda x: x.title, filtered_notes))

In [48]:
filtered_notes

In [77]:
for note in filtered_notes:
    print(note.id + note.title)

'16889e295a0.a03e63d51aa58e19Significant + Urgent'
'16889e3e4c7.938359362aa74d01Significant'
'16889e48842.80bf82edf6e55e71Urgent'
'1510551568605.2096093560Today'
'1702b89474c.a5e79116461ef293Shopping List'
'1584490858625.2086780820Not Significant + Not Urgent'
'1582912357250.1423760568Memory Bank for Reflect + Recollect'


In [51]:
note = list(filter(
    lambda x: has_attr_value(x, 'title', 'Significant + Urgent'), 
    notes))[0]
items = note.items

In [52]:
# print(dir(items[0]))

In [16]:
# remove tasks which are already completed
filtered_items = list(filter(lambda x: has_attr_value(x, 'checked', False), items))

In [17]:
# remove tasks which are children of some other task
filtered_items = list(filter(lambda x: has_attr_value(x, 'parent_item', None ), filtered_items))

In [18]:
filtered_tasks = [{'text': x.text, 'keep_id':x.id, 'children': x._subitems, 'created':x.timestamps._created} for x in filtered_items]

In [19]:
# filtered_tasks

In [20]:
for item in filtered_tasks:
    children = item.get('children').values()
    item['children'] = list(map(lambda x: {'text': x.text, 'keep_id': x.id, 'children':[], 'created': x.timestamps._created},filter(lambda x: has_attr_value(x, 'checked', False), children)))

In [21]:
filtered_tasks[0]

{'text': 'Read your algorithm, read other algorithms',
 'keep_id': '1584344571475.1329806704',
 'children': [],
 'created': datetime.datetime(2020, 3, 16, 7, 42, 52, 54000)}

In [22]:
filtered_items[0].timestamps.__dict__

{'_dirty': False,
 '_created': datetime.datetime(2020, 3, 16, 7, 42, 52, 54000),
 '_deleted': None,
 '_trashed': None,
 '_updated': datetime.datetime(2020, 3, 16, 7, 42, 52, 54000),
 '_edited': None}

In [23]:
filtered_items[4].__dict__

{'_dirty': False,
 'parent': <gkeepapi.node.List at 0x7fe3fb3d36d8>,
 'id': '16889e8da53.ad0f7e575ef82686',
 'server_id': '1-r3n6Fx_SlMaR81xk-VaRnTrOi_cqS3FkoB-SJAhBcb5n8eiG1GtcYxHO3K7lHUH_IUA',
 'parent_id': '16889e295a0.a03e63d51aa58e19',
 'type': <NodeType.ListItem: 'LIST_ITEM'>,
 '_sort': '-389267117550330',
 '_version': '2',
 '_text': 'Job',
 '_children': {},
 'timestamps': <gkeepapi.node.NodeTimestamps at 0x7fe3fab55048>,
 'settings': <gkeepapi.node.NodeSettings at 0x7fe3fab55080>,
 'annotations': <gkeepapi.node.NodeAnnotations at 0x7fe3fab550b8>,
 '_moved': False,
 'parent_item': None,
 'parent_server_id': None,
 'super_list_item_id': None,
 'prev_super_list_item_id': None,
 '_subitems': {'1581546468253.1438110809': <gkeepapi.node.ListItem at 0x7fe3fab48b70>,
  '1581546474520.1165161214': <gkeepapi.node.ListItem at 0x7fe3fab48c50>},
 '_checked': False}

In [24]:
# so we have the tasks!
# now we have to exec the task add command get the task id, use that to link children!

In [25]:
# for loop
# add task_warrior id in dict
# need to get that by parsing the subprocess.PIPE.etc

In [26]:
for task in filtered_tasks:
    print(task)

{'children': [],
 'created': datetime.datetime(2020, 3, 16, 7, 42, 52, 54000),
 'keep_id': '1584344571475.1329806704',
 'text': 'Read your algorithm, read other algorithms'}
{'children': [],
 'created': datetime.datetime(2020, 3, 13, 23, 13, 50, 109000),
 'keep_id': '1584141228696.1680011852',
 'text': 'Mail '}
{'children': [],
 'created': datetime.datetime(2020, 3, 17, 6, 37, 53, 873000),
 'keep_id': '1584427066200.2085338598',
 'text': 'inThe.am + streak --> zapier? what else?'}
{'children': [{'children': [],
               'created': datetime.datetime(2020, 2, 27, 8, 13, 47, 267000),
               'keep_id': '1582791225563.1845751236',
               'text': 'Callbacks and review of what you wrote previously'},
              {'children': [],
               'created': datetime.datetime(2020, 2, 27, 8, 14, 8, 440000),
               'keep_id': '1582791246559.2029377938',
               'text': 'An easier way to move through the form (as sometimes I '
                       'remember 

In [96]:
keep._keep_api.changes(keep._keep_version)['nodes']

[{'kind': 'notes#node',
  'id': '16889e3e4c7.938359362aa74d01',
  'serverId': '18CJsniDTG2S2Q5dX-3L-nvzfRAr8wtTpS40WSe0KiwTbKZjj7ffrggb2A5QiDXRiluIL',
  'parentId': 'root',
  'type': 'LIST',
  'timestamps': {'kind': 'notes#timestamps',
   'created': '2019-01-26T11:20:54.981Z',
   'updated': '2020-03-21T06:21:05.265Z',
   'trashed': '1970-01-01T00:00:00.000Z',
   'userEdited': '2020-03-21T06:21:04.777Z',
   'recentSharedChangesSeen': '2020-03-20T23:34:19.206Z'},
  'title': 'Significant',
  'nodeSettings': {'newListItemPlacement': 'BOTTOM',
   'checkedListItemsPolicy': 'GRAVEYARD',
   'graveyardState': 'COLLAPSED'},
  'isArchived': False,
  'isPinned': True,
  'color': 'CERULEAN',
  'sortValue': '3122659328',
  'annotationsGroup': {'kind': 'notes#annotationsGroup'},
  'lastModifierEmail': 'achalagarwal.01@gmail.com',
  'moved': '1'},
 {'kind': 'notes#node',
  'id': '1699a15f5ff.8be52ddb7a476385',
  'serverId': '1dQZOBmCTJIir4_6m133Foj4NGQ2m4kARxm1_kBKV9XMb2oSNKu4lZXPlqsUBhK0ZE5w0',
  'pa

In [32]:
keep.sync(True)

In [56]:
# 

In [69]:
# now I need to perform incremental updates on my taskwarrior entries
# so for that I need to be able to extract task id from annotations!

# the incremental updates are limited to checked/unchecked and/or adding children, because they lead to changes in parents, however will it be tricky to get parents and children updates when we need the task id?

# i dont know this right now because of obvious reasons
# firstly i dont know in what form the incremental updates arrive 
# if they are of the form, a child has been added, heres the new child, 
# dont have to use a dictionary for efficiency right now as we have enough compute power






In [ ]:
def perform_incremental_update(nodes, current):
    # nodes are the latest updates that have been fetched from the api
    # current is the current set of filtered tasks
    # 